## SRJ - H2O

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    print('Singles are', singles)
    print('Doubles are', doubles)
    print('In adapt, singles are', len(singles))
    print('In adapt, doubles are', len(doubles))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    print('Null state is', null_state)

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value


        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization



        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)


        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H

## So if you want the state, return the ostate and not states


import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import scipy
configd = []

#elec: 8, spin-orb: 12
def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count = count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                    
            #print('Config singles', config)
            #print('Len of singles in excited states', len(config))    
            list1.append(config)
            config=[]
    print('Len of singles', len(list1))
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            print('I am changing orb-elec')
            count1=orb-elec
            count2=orb-elec
            print('Doubles, count1', count1)
            print('Doubles, count2', count2)
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        #print('Config', config)
                        list1.append(config)
                        print('lenght of total exc:', len(list1))
                        config=[]
    return list1

def ee_exact(symbols, coordinates, active_electrons, active_orbitals ,params,ash_excitation, shots=0):
    print('The no of active electrons:', active_electrons)
    print('The no of active orbitals:', active_orbitals)
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals)
    hf_state = qml.qchem.hf_state(active_electrons, qubits)
    print('HF state:', hf_state)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    wires=range(qubits)
    

    null_state = np.zeros(qubits,int)
    print('Null state is', null_state)
    list1 = inite(active_electrons,qubits)
    print('The list1 :', list1)
    print('Len of list1', len(list1))
    values =[]
    for t in range(1):
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
        else:

            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            #print('What is going  as hf_State:', hf_state)
            qml.BasisState(hf_state, wires=range(qubits))
            for w in occ:
                qml.X(wires=w)
            #Going to include excitations here
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, hf_state, ash_excitation):
            #print('What is going  as hf_State:', hf_state)
            #print('The occ1', occ1)
            #print('The occ2', occ2)
            qml.BasisState(hf_state, wires=range(qubits))
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        #print('Hadamard applied to',v)
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    #print('Checking if its printing anything')
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        #final M matrix
        M = np.zeros((len(list1),len(list1)))
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        print("diagonal parts done")
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, null_state, ash_excitation)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        print("off diagonal terms done")
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values

symbols  = [ 'H', 'O', 'H']
print('H2O-0.6A-GS+ES-DBS')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,-0.7578*r_bohr, 0.5851*r_bohr], [0.0, 0.0, 0.0*r_bohr], [0.0,0.7578*r_bohr, 0.5851*r_bohr]])

print(coordinates)

electrons = 10  # 7 from N and 3 from H
orbitals = 14 #Spin - Prince said its spin  
charge = 0

active_electrons = 8
active_orbitals = 6 #Spatial orbitals


gs_state, params, ash_excitation, qubits, H = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)



eig = ee_exact(symbols, coordinates,active_electrons,active_orbitals,params,ash_excitation)
print('exact eigenvalues:\n', eig)


H2O-0.6A-GS+ES-DBS
[[ 0.         -1.43203435  1.10567868]
 [ 0.          0.          0.        ]
 [ 0.          1.43203435  1.10567868]]
Using active space, check if you change the H accordingly
core orbitals: [0]
active orbitals: [1, 2, 3, 4, 5, 6]
-70.06191369238309 * I([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]) + 1.168809905801588 * Z(0) + 0.06056453175842947 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.06056453175842947 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + -0.13899551034929608 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8)) + -0.13899551034929608 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8)) + 0.8132206835400954 * Z(2) + 0.1251663072369635 * (Z(0) @ Z(2)) + -0.19173645110792875 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + -0.19173645110792875 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + -0.024502168101866333 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + -0.024502168101866333 * 

/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Highest gradient excitation is [2, 3, 10, 11]
Current parameters: [0.16457859]
Current cost: -74.97548391892154

Current parameters: [0.16457869]
Current cost: -74.97548391892155

Final updated parameters: [0.16457869]
Final cost: -74.97548391892155
The params after GS is [0.16457869]
Ash excitation after gs state: [[2, 3, 10, 11]]
The no of active electrons: 8
The no of active orbitals: 6
HF state: [1 1 1 1 1 1 1 1 0 0 0 0]
Null state is [0 0 0 0 0 0 0 0 0 0 0 0]
Len of singles 16
I am changing orb-elec
Doubles, count1 4
Doubles, count2 4
lenght of total exc: 17
lenght of total exc: 18
lenght of total exc: 19
lenght of total exc: 20
I am changing orb-elec
Doubles, count1 4
Doubles, count2 4
lenght of total exc: 21
I am changing orb-elec
Doubles, count1 4
Doubles, count2 4
lenght of total exc: 22
lenght of total exc: 23
lenght of total exc: 24
lenght of total exc: 25
I am changing orb-elec
Doubles, count1 4
Doubles, count2 4
lenght of total exc: 26
I am changing orb-elec
Doubles, count

In [ ]:
coordinates = np.array([[0.0,-0.7578*r_bohr, 0.5851*r_bohr], [0.0, 0.0, 0.0*r_bohr], [0.0,0.7578*r_bohr, 0.5851*r_bohr]])

In [ ]:
-74.96291283980221

[-74.97548391892155, -75.07924369+0.00000000e+00j, -74.97548392+0.00000000e+00j,
        -74.97548392+0.00000000e+00j, -74.97548392+0.00000000e+00j,
        -74.58445473+0.00000000e+00j, -74.53208345+0.00000000e+00j,
        -74.49098491+0.00000000e+00j, -74.45477002+0.00000000e+00j,
        -74.4319143 +0.00000000e+00j, -74.39645411+0.00000000e+00j,
        -74.34593057+0.00000000e+00j, -74.30672155+0.00000000e+00j,
        -74.29222006+0.00000000e+00j, -74.20962382+0.00000000e+00j,
        -74.17484711+0.00000000e+00j, -73.99482236+0.00000000e+00j,
        -73.94909169+0.00000000e+00j, -73.93732471+0.00000000e+00j,
        -73.9226538 +0.00000000e+00j, -73.89476384+0.00000000e+00j,
        -73.86882073+0.00000000e+00j, -73.86481813+0.00000000e+00j,
        -73.84940222+0.00000000e+00j, -73.83952149+0.00000000e+00j,
        -73.83406465+0.00000000e+00j, -73.78989199+0.00000000e+00j,
        -73.78132386+0.00000000e+00j, -73.76277762+0.00000000e+00j,
        -73.75030474+0.00000000e+00j, -73.73914282-1.69655972e+00j,
        -73.73914282+1.69655972e+00j, -73.71027492+0.00000000e+00j,
        -73.69641631+0.00000000e+00j, -73.69533918+0.00000000e+00j,
        -73.67890489+0.00000000e+00j, -73.65603077-8.57381122e-03j,
        -73.65603077+8.57381122e-03j, -73.64275777+0.00000000e+00j,
        -73.62893712+0.00000000e+00j, -73.61667756+0.00000000e+00j,
        -73.58917738+0.00000000e+00j, -73.58196013-3.86714001e-03j,
        -73.58196013+3.86714001e-03j, -73.57183943+0.00000000e+00j,
        -73.57056143+0.00000000e+00j, -73.54395057+0.00000000e+00j,
        -73.53779683+0.00000000e+00j, -73.49808929+0.00000000e+00j,
        -73.48987293+0.00000000e+00j, -73.48145223-1.12279730e+01j,
        -73.48145223+1.12279730e+01j, -73.46000335+0.00000000e+00j,
        -73.45370035-1.64432854e-02j, -73.45370035+1.64432854e-02j,
        -73.4220053 -1.36419287e-02j, -73.4220053 +1.36419287e-02j,
        -73.41764411+0.00000000e+00j, -73.40612484+0.00000000e+00j,
        -73.39924687-3.53315130e-04j, -73.39924687+3.53315130e-04j,
        -73.38196513+0.00000000e+00j, -73.37017393-2.44722381e-02j,
        -73.37017393+2.44722381e-02j, -73.35892834+0.00000000e+00j,
        -73.35009444+0.00000000e+00j, -73.29296204+0.00000000e+00j,
        -73.24988643+0.00000000e+00j, -73.24285002+0.00000000e+00j,
        -73.21843895+0.00000000e+00j, -73.19942118+0.00000000e+00j,
        -73.16244298+0.00000000e+00j, -73.12368851+0.00000000e+00j,
        -73.08899608+0.00000000e+00j, -73.05841021-1.48132243e+00j,
        -73.05841021+1.48132243e+00j, -73.04026202-2.07375231e+00j,
        -73.04026202+2.07375231e+00j, -73.01878386+0.00000000e+00j,
        -72.98285811+0.00000000e+00j, -72.97283809+0.00000000e+00j,
        -72.95081593-1.62340020e-02j, -72.95081593+1.62340020e-02j,
        -72.93530261+0.00000000e+00j, -72.89565475-8.96449733e-02j,
        -72.89565475+8.96449733e-02j, -72.89132002+0.00000000e+00j,
        -72.86332103+0.00000000e+00j, -72.85955137-6.10680080e-03j,
        -72.85955137+6.10680080e-03j, -72.84564029+0.00000000e+00j,
        -72.8097094 +0.00000000e+00j, -72.79966983+0.00000000e+00j,
        -72.76468044-5.69320724e-02j, -72.76468044+5.69320724e-02j,
        -72.74209578+0.00000000e+00j, -72.70867685+0.00000000e+00j,
        -72.69339519+0.00000000e+00j, -72.66905899+0.00000000e+00j,
        -72.63922142+0.00000000e+00j, -72.62700263+0.00000000e+00j,
        -72.60058705+0.00000000e+00j, -72.56956278+0.00000000e+00j,
        -72.38226712+0.00000000e+00j, -72.37912674+0.00000000e+00j,
        -72.20228573+0.00000000e+00j, -72.11832132+0.00000000e+00j,
        -71.84598616+0.00000000e+00j, -71.84384653+0.00000000e+00j]

In [ ]:
from pyscf import gto, scf, mcscf
import numpy as np
#coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.20*r_bohr], [1.20*r_bohr,0.0,-0.336000*r_bohr],[-0.475176*r_bohr, -0.823029*r_bohr, -0.336000*r_bohr]])

# Define the molecule
mol = gto.M(
    atom='H 0 -0.7578 0.5851; O 0 0 0; H 0 0.7578 0.5851',
    basis='sto-3g',
    verbose=1  # Optional: Adjusts verbosity for output
)
 
# Perform Restricted Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()
 
# Get the HF energy
hf_energy = mf.e_tot
print("Hartree-Fock Ground State Energy:", hf_energy)
 
# Define the active space
ncas = 6  # Number of active orbitals - spatial it seems
nelecas = 8  # Number of electrons in the active space
 
# Set up the CASCI/CASSCF solver
mycas = mcscf.CASCI(mf, ncas, nelecas)
 
 
mycas.fcisolver.nroots=93
 
# Compute the CASCI/CASSCF energy
e_casci = mycas.kernel()
 
# Output the results
print("CASCI/CASSCF Ground State Energy:", e_casci)
print("Hartree-Fock Ground State Energy:", hf_energy)